<a href="https://colab.research.google.com/github/Venkat3103/fantasy-football/blob/main/EPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
!pip install pulp

In [115]:
import pandas as pd
from scipy.optimize import linprog
from scipy.stats import linregress
from pulp import *

In [107]:
from google.colab import files
uploaded = files.upload()

Saving epldata_final.csv to epldata_final (3).csv


In [108]:
import io
df=pd.read_csv(io.BytesIO(uploaded['epldata_final.csv']))

In [116]:
df

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing,pos_detail
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0,striker
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0,midfielder
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2.0,Czech Republic,0,6,1,1,0,goalkeeper
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1.0,England,0,4,1,1,0,striker
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2.0,France,0,4,1,1,0,defender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Edimilson Fernandes,West+Ham,21,CM,2,5.0,288,4.5,0.40%,38,2.0,Switzerland,0,1,20,0,1,midfielder
457,Arthur Masuaku,West+Ham,23,LB,3,7.0,199,4.5,0.20%,34,4.0,Congo DR,0,2,20,0,1,defender
458,Sam Byram,West+Ham,23,RB,3,4.5,198,4.5,0.30%,29,1.0,England,0,2,20,0,0,defender
459,Ashley Fletcher,West+Ham,21,CF,1,1.0,412,4.5,5.90%,16,1.0,England,0,1,20,0,1,striker


In [117]:
#to be changed as points per match 
#df['ppc']=df['fpl_points']/df['fpl_value']
#df.head(50)

In [118]:
df['pos_detail']=0
df['pos_detail'][(df['position']=="LW") | (df['position']=="RW") | (df['position']=="SS") | (df['position']=="CF")]="striker"
df['pos_detail'][(df['position']=="CB") | (df['position']=="RB") | (df['position']=="LB")]="defender"
df['pos_detail'][(df['position']=="LM") | (df['position']=="RM") | (df['position']=="DM") | (df['position']=="CM") | (df['position']=="AM")]="midfielder"
df['pos_detail'][df['position']=="GK"]="goalkeeper"
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing,pos_detail
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0,striker
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0,midfielder
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2.0,Czech Republic,0,6,1,1,0,goalkeeper
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1.0,England,0,4,1,1,0,striker
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2.0,France,0,4,1,1,0,defender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Edimilson Fernandes,West+Ham,21,CM,2,5.0,288,4.5,0.40%,38,2.0,Switzerland,0,1,20,0,1,midfielder
457,Arthur Masuaku,West+Ham,23,LB,3,7.0,199,4.5,0.20%,34,4.0,Congo DR,0,2,20,0,1,defender
458,Sam Byram,West+Ham,23,RB,3,4.5,198,4.5,0.30%,29,1.0,England,0,2,20,0,0,defender
459,Ashley Fletcher,West+Ham,21,CF,1,1.0,412,4.5,5.90%,16,1.0,England,0,1,20,0,1,striker


pos_detail,fpl_val,fpl_points,club


In [119]:
avc=df[['name','pos_detail','fpl_value','fpl_points','club']][(df['club']=="Arsenal") | (df['club']=="Chelsea")].copy()
data=avc
data.reset_index(inplace=True,drop=True)
data

,name,pos_detail,fpl_value,fpl_points,club
0,Alexis Sanchez,striker,12.0,264,Arsenal
1,Mesut Ozil,midfielder,9.5,167,Arsenal
2,Petr Cech,goalkeeper,5.5,134,Arsenal
3,Theo Walcott,striker,7.5,122,Arsenal
4,Laurent Koscielny,defender,6.0,121,Arsenal
5,Hector Bellerin,defender,6.0,119,Arsenal
6,Olivier Giroud,striker,8.5,116,Arsenal
7,Nacho Monreal,defender,5.5,115,Arsenal
8,Shkodran Mustafi,defender,5.5,90,Arsenal
9,Alex Iwobi,striker,5.5,89,Arsenal


In [120]:
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['fpl_points'][i] for i in range(data.shape[0])} 
cost = {str(i): data['fpl_value'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
arsenal = {str(i): 1 if data['club'][i] == 'Arsenal' else 0 for i in range(data.shape[0])}
chelsea = {str(i): 1 if data['club'][i] == 'Chelsea' else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([arsenal[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from Arsenal"
prob += lpSum([chelsea[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from Bournemouth"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [121]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [122]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue
selection
#prob.variables

{0: 1.0,
 1: 1.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 1.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 1.0,
 37: 0.0,
 38: 1.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0}

In [123]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]
data

,name,pos_detail,fpl_value,fpl_points,club,integer_programming
0,Alexis Sanchez,striker,12.0,264,Arsenal,1.0
1,Mesut Ozil,midfielder,9.5,167,Arsenal,1.0
2,Petr Cech,goalkeeper,5.5,134,Arsenal,1.0
3,Theo Walcott,striker,7.5,122,Arsenal,0.0
4,Laurent Koscielny,defender,6.0,121,Arsenal,1.0
5,Hector Bellerin,defender,6.0,119,Arsenal,0.0
6,Olivier Giroud,striker,8.5,116,Arsenal,0.0
7,Nacho Monreal,defender,5.5,115,Arsenal,0.0
8,Shkodran Mustafi,defender,5.5,90,Arsenal,0.0
9,Alex Iwobi,striker,5.5,89,Arsenal,0.0


In [124]:
XI = data[data['integer_programming'] == 1.0]
XI
TOP2 = XI['fpl_points'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.fpl_points==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.fpl_points==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['fpl_points'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['fpl_value'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total points:{}, cost:£{}, Captain:{}, Vice-Captain:{}".format(TOTAL_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

#VICECAPTAIN.iloc[0]
#CAPTAIN.iloc[0]
#CAPTAIN_POINTS
#VICECAPTAIN_POINTS

Total points:2233.0, cost:£87.0, Captain:Alexis Sanchez, Vice-Captain:Eden Hazard


In [125]:
(XI[['name','pos_detail','fpl_points','fpl_value','club']]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

,name,pos_detail,fpl_points,fpl_value,club
0,Laurent Koscielny,defender,121,6.0,Arsenal
1,Gary Cahill,defender,178,6.5,Chelsea
2,Marcos Alonso Mendoza,defender,177,7.0,Chelsea
3,Cesar Azpilicueta,defender,170,6.5,Chelsea
4,Petr Cech,goalkeeper,134,5.5,Arsenal
5,Mesut Ozil,midfielder,167,9.5,Arsenal
6,Cesc Fabregas,midfielder,121,7.0,Chelsea
7,Victor Moses,midfielder,105,6.5,Chelsea
8,Alexis Sanchez,striker,264,12.0,Arsenal
9,Eden Hazard,striker,224,10.5,Chelsea
